German Traffic Sign Dataset merupakan benchmark dataset untuk klasifikasi gambar multi kelas. Terdapat satu rambu lalu lintas pada masing-masing gambar, sehingga dataset ini disebut juga sebagai single-image atau gambar tunggal. Dataset ini pertama kali diluncurkan dalam perlombaan (challenge) yang diselenggarakan oleh the International Joint Conference on Neural Networks (IJCNN) pada tahun 2011. 

Dataset ini memiliki komponen sebagai berikut:

Data untuk permasalahan klasifikasi gambar tunggal dengan banyak kelas.
Terdiri dari 43 kelas, artinya ada 43 jenis rambu lalu lintas.
Memiliki total lebih dari 50.000 gambar .
Data diambil dari foto rambu yang sebenarnya (bukan sintetis).
Data yang akan kita gunakan dalam bentuk file pickle yang dibuat dengan modul Python bernama pickle. Ia digunakan untuk mengubah objek Python menjadi representasi byte untuk disimpan pada storage atau dipindahkan melalui jaringan. Hal ini memungkinkan objek untuk disimpan atau ditransmisikan dengan mudah tanpa mengubah data ke format lain terlebih dahulu.
https://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPool2D
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Load Data

In [ ]:
## Load the data
training_file = "/content/drive/MyDrive/Dataset/German Traffic Sign/train.p"
testing_file = "/content/drive/MyDrive/Dataset/German Traffic Sign/test.p"   
 
# Open and load the training file 
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
 
# Open and load the testing file
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
print("Data loaded")

# Ekplorasi Data
Tahapan eksplorasi data yang pertama adalah membaca file signnames.csv. File ini berisi id dan nama kelas dataset traffic sign. Mari implementasikan kode berikut.

In [ ]:
## Buat pandas dataframe untuk load data csv
## File csv ini berupa ClassId dan SignName
 
sign_name_df = pd.read_csv('/content/drive/MyDrive/Dataset/German Traffic Sign/signnames.csv')
SIGN_NAMES = sign_name_df.SignName.values
sign_name_df.set_index('ClassId', inplace=True)
sign_name_df.head(10)

In [ ]:
# Definisikan fitur dan label untuk data training
X, y = train['features'], train['labels']
 
# Mengubah lists menjadi numpy arrays
data = np.array(X)
labels = np.array(y)
print(data.shape, labels.shape)
 
# Definisikan fitur dan label untuk data testing
X_test, y_test = test['features'], test['labels']
 
# Mengubah lists menjadi numpy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_test.shape, y_test.shape)

In [ ]:
# Split training data into train and val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
n_labels = np.unique(y_train).size
def hist_data(y_data, title=None, ax=None, **kwargs):
    if not ax :
        fig = plt.figure()
        ax = fig.add_subplot(111)
    ax.hist(y_data, np.arange(-0.5, n_labels+1.5), stacked=True, **kwargs)
    ax.set_xlim(-0.5, n_labels-0.5)
    if 'label' in kwargs : ax.legend()
    if title : ax.set_title(title)
        
fig,ax = plt.subplots(1,3, figsize=(20,5))
hist_data(y_train, title='Distribusi kelas pada data training', ax=ax[0])
hist_data(y_val, title='Distribusi kelas pada data validasi', ax=ax[1], color='black')
hist_data(y_test, title='Distribusi kelas pada data test', ax=ax[2], color='grey')

![Image Data](distribusi_traffic_data.png)

In [ ]:
# Konversi label dengan teknik one hot encoding
from tensorflow.keras.utils import to_categorical
 
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

## Modelling

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.96):
      print("\nAkurasi telah mencapai >96%. Stop training!")
      self.model.stop_training = True
callbacks = myCallback()

![Modelling Architecture](modelling.png)

Mari kita ingat kembali sebuah arsitektur Convolutional Network sederhana yang memiliki arsitektur lapisan (layer) Input-Convolutional-RELU-Pooling-FC. Berikut uraiannya:

1. apisan Input akan menampung gambar sebagai array 3D dari nilai piksel.

2. Lapisan Convolutional (konvolusi) akan menghitung perkalian (dot product) antara kernel dan sub-array dari gambar input yang berukuran sama dengan kernel. Sebagai contoh pada gambar berikut kita menggunakan kernel berukuran 3x3 dan melakukan proses perkalian antara kernel dan sub-array pada gambar masukan.

![Perkalian Kernel pada Konvolusi](perkalian_konvolusi.gif)

Matriks kernel akan melintasi setiap piksel pada gambar secara berurutan dari kiri ke kanan dan atas ke bawah. Ia menghitung nilai output untuk setiap piksel yang dilintasi kernel dengan teknik perkalian matriks dot product. Kemudian, semua nilai yang dihasilkan dari proses perkalian dot product ini akan menjadi nilai piksel tunggal dari gambar keluaran. Proses ini diulang sampai seluruh gambar input terlewati oleh kernel.

    RELU atau Rectified Linear Activation Unit merupakan fungsi aktivasi linear untuk jaringan saraf tiruan. Ia telah menjadi fungsi aktivasi default karena model yang menggunakannya mencapai kinerja lebih baik dengan cepat.

    Ingatlah, fungsi aktivasi digunakan untuk memetakan hasil penjumlahan antara bobot dengan masukan (disebut sebagai weighted sum) menjadi nilai tertentu. Fungsi ini memungkinkan perceptron dapat menyesuaikan pola untuk data non linear.

    Nah, jika weighted sum ini bernilai positif, RELU akan mengembalikan nilainya secara langsung. Sebaliknya, jika nilainya negatif, RELU akan menghasilkan nilai nol. RELU didefinisikan sebagai:

    f(x) = max(0, x), dengan x adalah jaringan saraf masukan (weighted sum).


3. Lapisan Pooling akan melakukan downsampling pada shape suatu gambar sehingga mengakibatkan pengurangan dimensi gambar. Tujuannya adalah agar data komputasi yang dibutuhkan untuk memproses citra menjadi berkurang.

Pooling layer terdiri dari dua jenis, average pooling dan max pooling. Pada max pooling, setiap area dengan luas piksel tertentu akan diambil satu buah piksel yang memiliki nilai tertinggi. Sementara pada average pooling, nilai yang diambil adalah nilai rata-rata dari suatu area kernel. Untuk lebih jelasnya, mari kita perhatikan gambar berikut.

![Pooling](pooling.png)

    Lapisan pooling juga bertindak sebagai pengendali noise (gangguan). Namun, max pooling memiliki kinerja lebih baik dibanding dengan average pooling. Oleh karena itu, max pooling lebih sering digunakan pada proses training CNN. Kita juga akan menggunakannya dalam proyek ini.

4. Lapisan FC (Fully-Connected) akan menghitung skor kelas untuk setiap kategori klasifikasi. Lapisan ini sama dengan jaringan saraf di mana setiap neuron terhubung ke semua neuron pada lapisan secara berurutan. Output akhir dihitung menggunakan softmax yang memberikan probabilitas setiap kelas untuk fitur yang diberikan.

Pada proyek ini, kita menggunakan arsitektur 2 convolutional layer dan 1 fully connected layer. Untuk lebih jelasnya, mari kita bahas setiap layernya.

Pertama, inisialisasi model Sequential dan tentukan input shape untuk model. Set layer ini menggunakan kernel berukuran 5x5 untuk mempelajari fitur pada data traffic sign. Input masukan kita sesuaikan dengan shape X_train yaitu sebuah gambar RGB berukuran 32 pixels (32, 32, 3). Fungsi aktivasi yang akan kita gunakan adalah RELU, seperti yang telah dijelaskan sebelumnya. Max Pooling layer juga kita tambahkan untuk mengurangi dimensi.

Di sini, kita juga menerapkan dropout. Dropout merupakan salah satu hyperparameter yang bertujuan untuk mengurangi overfitting. Ia bekerja dengan memutus atau menjatuhkan (drop) unit selama proses training neural network sehingga layer atau lapisannya menjadi tidak aktif. Berdasarkan dokumentasi dari Keras, argumen rate untuk dropout adalah tipe data float antara 0 dan 1, yang merupakan bagian dari unit yang akan di-drop. 

Jika kita set dropout rate = 0.25, artinya, kita akan melepas 25% dari layer dan membuatnya menjadi tidak aktif selama proses training.

![set layer](set_layer.gif)

Berikut set layer kita yang pertama. Pada arsitektur ini, set parameter layer sebagai berikut:
- Ukuran filter untuk proses konvolusi=32
- Ukuran kernel=(5,5)
- Fungsi aktivasi RELU
- Pooling yang kita gunakan adalah Maxpool dengan ukuran 2,2
- Dropout rate sebesar 0.25

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

### Layer 2

In [ ]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

### Fully Connected Layer

In [ ]:
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

### Training

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 25
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val), callbacks=[callbacks])
model.save("my_model.h5")

In [ ]:
# Plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
 
# Plotting graphs for loss
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()